In [1]:
import sys
import string
import itertools
from collections import Counter, defaultdict
import re

from pathlib import Path
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%load_ext line_profiler

In [3]:
data = Path('../data/day_08.txt').read_text()

In [4]:
inps = [k.split(' | ') for k in data.strip().splitlines()]

In [5]:
len_to_num = {
    2: 1,
    3: 7,
    4: 4,
    7: 8
}

In [6]:
def part_a():
    exists = 0
    for src, out in inps:
        for c in out.split():
            if len(c.strip()) in len_to_num:
                exists += 1

    return exists

print(part_a())
%timeit part_a()

375
104 µs ± 1.82 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
%lprun -f part_a part_a()

Timer unit: 1e-06 s

Total time: 0.001595 s
File: <ipython-input-6-506271491892>
Function: part_a at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def part_a():
     2         1          2.0      2.0      0.1      exists = 0
     3       201         93.0      0.5      5.8      for src, out in inps:
     4      1000        532.0      0.5     33.4          for c in out.split():
     5       800        701.0      0.9     43.9              if len(c.strip()) in len_to_num:
     6       375        267.0      0.7     16.7                  exists += 1
     7                                           
     8         1          0.0      0.0      0.0      return exists

In [8]:
def solve(signals, output):
    mapped = {}
    length_to_possibles = defaultdict(list)

    TOP_HORI = MID_HORI = BOT_HORI = TOP_LEFT = TOP_RIGHT = BOTTOM_LEFT = BOTTOM_RIGHT = None

    for c in signals:
        if len(c) in len_to_num: # identifies uniques by length i.e. for 1,4,7,8 (part 1)
            mapped[len_to_num[len(c)]] = set(c)
        else:
            length_to_possibles[len(c)].append(set(c))

    TOP_HORI = next(iter(mapped[7] - mapped[1])) # 1 and 7 difference = top horizontal
    RIGHTS = mapped[7] & mapped[1] # 1 and 7 commons = the right verticals
    HORIS = set.intersection(*length_to_possibles[5]) # 2,3,5 commons = the horizontals

    to_remove = []
    for v in length_to_possibles[6]: # iterate over 6 lengths i.e. 0,6,9 possible
        if not RIGHTS.issubset(v): # only 6 contains both right verticals
            mapped[6] = v
            to_remove.append(v)
        elif not HORIS.issubset(v): # only 0 does not contain all the horizontals
            mapped[0] = v
            MID_HORI = next(iter(HORIS - v)) # as a consequence, mid horizontal can be identified
            to_remove.append(v)
        else: # otherwise it is 9
            mapped[9] = v

    TOP_LEFT = next(iter(mapped[4] - RIGHTS.union({MID_HORI}))) # top left = [4] - the rest 

    for v in length_to_possibles[5]: # iterate over 5 lengths i.e. 2,3,5 possible
        if RIGHTS.issubset(v): # only 3 contains both right verticals
            mapped[3] = v
        elif TOP_LEFT in v: # only 5 contains top left vertical
            mapped[5] = v
            BOTTOM_RIGHT = next(iter(RIGHTS & v))
            TOP_RIGHT = next(iter(RIGHTS - {BOTTOM_RIGHT}))
        else: # otherwise 2
            mapped[2] = v
            BOTTOM_LEFT = next(iter(v - HORIS.union({TOP_RIGHT})))

    BOT_HORI = next(iter(HORIS - {TOP_HORI, MID_HORI}))


    out_to_num = {''.join(sorted(v)): k for k,v in mapped.items()}
    final = 0

    for num in output:
        final = final * 10 + out_to_num[''.join(sorted(num))]
        
    return final

In [9]:
solve('acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab'.split(), 'cdfeb fcadb cdfeb cdbaf'.split())

5353

In [10]:
def part_b(inps):
    s = 0
    for src, out in inps:
        s += solve(src.split(), out.split())
        
    return s
print(part_b(inps))
%timeit part_b(inps)

1019355
3.24 ms ± 87.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%lprun -f part_b -f solve part_b(inps)

Timer unit: 1e-06 s

Total time: 0.030062 s
File: <ipython-input-10-a84760f6e48e>
Function: part_b at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def part_b(inps):
     2         1          1.0      1.0      0.0      s = 0
     3       201        141.0      0.7      0.5      for src, out in inps:
     4       200      29920.0    149.6     99.5          s += solve(src.split(), out.split())
     5                                                   
     6         1          0.0      0.0      0.0      return s

Total time: 0.017893 s
File: <ipython-input-8-d6a9915a0326>
Function: solve at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def solve(signals, output):
     2       200        153.0      0.8      0.9      mapped = {}
     3       200        271.0      1.4      1.5      length_to_possibles = defaultdict(list)
     4                    